In [ ]:
%matplotlib notebook

In [ ]:
%matplotlib inline

In [ ]:
from adler.dataclasses.AdlerPlanetoid import AdlerPlanetoid
from adler.dataclasses.AdlerData import AdlerData
from adler.science.PhaseCurve import PhaseCurve
from adler.utilities.plotting_utilities import plot_errorbar

# from adler.utilities.science_utilities import get_df_obs_filt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u
from astropy.modeling.fitting import SLSQPLSQFitter

In [ ]:
# ssObjectId of object to analyse
# ssoid = "8268570668335894776" # NEO
# ssoid = "6098332225018"  # good MBA test object
ssoid = "5334524274243260416"

In [ ]:
# fname = "../tests/data/testing_database.db"
# fname = "/Users/jrobinson/lsst-adler/notebooks/gen_test_data/adler_demo_testing_database.db"
# planetoid = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=fname)
planetoid = AdlerPlanetoid.construct_from_RSP(ssoid)

In [ ]:
resample = 100

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

adler_cols = AdlerData(ssoid, planetoid.filter_list)

for filt in ["g", "r"]:
    # get observations and define a phase curve model to fit

    sso = planetoid.SSObject_in_filter(filt)
    obs = planetoid.observations_in_filter(filt)

    H = sso.H
    G12 = sso.G12
    pc = PhaseCurve(H=H * u.mag, phase_parameter_1=G12, model_name="HG12_Pen16")

    alpha = (np.array(getattr(obs, "phaseAngle")) * u.deg,)
    mag_err = np.array(getattr(obs, "magErr")) * u.mag
    red_mag = np.array(getattr(obs, "reduced_mag")) * u.mag

    pc_fit = pc.FitModel(
        np.array(getattr(obs, "phaseAngle")) * u.deg,
        red_mag,
        mag_err,
    )
    pc = pc.InitModelSbpy(pc_fit)

    #     adler_cols.populate_phase_parameters(filt,**pc.ReturnModelDict())

    # add this phase curve to the figure
    fig = plot_errorbar(planetoid, filt_list=[filt], fig=fig)
    ax1 = fig.axes[0]
    alpha_fit = np.linspace(0, np.amax(obs.phaseAngle)) * u.deg
    ax1.plot(
        alpha_fit.value,
        pc.ReducedMag(alpha_fit).value,
        label="{}: H={:.2f}, G12={:.2f}".format(filt, pc.H, pc.phase_parameter_1),
    )
    ax1.legend()

    mc_models = []
    for i in range(resample):
        _red_mag = np.random.normal(loc=np.array(red_mag), scale=np.array(mag_err)) * u.mag
        ax1.scatter(alpha[0].value, _red_mag.value, s=1, c="k", alpha=0.1, zorder=5)

        _pc_fit = pc.FitModel(
            np.array(getattr(obs, "phaseAngle")) * u.deg,
            _red_mag,
        )
        _pc = pc.InitModelSbpy(_pc_fit)
        ax1.plot(
            alpha_fit.value,
            _pc.ReducedMag(alpha_fit).value,
            c="k",
            alpha=0.1,
            zorder=0,
            label="{}: H={:.2f}, G12={:.2f}".format(filt, _pc.H, _pc.phase_parameter_1),
        )

        mc_models.append(_pc)

    break

ax1.invert_yaxis()

plt.show()

In [ ]:
mag_err

In [ ]:
np.median(mag_err)

In [ ]:
mc_models[0].H, mc_models[0].phase_parameter_1

In [ ]:
abs_mag = np.array([x.H.value for x in mc_models])
phase_param = np.array([x.phase_parameter_1 for x in mc_models])

In [ ]:
phase_param

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 2)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])

for x, ax, ax_lab in zip([abs_mag, phase_param], [ax1, ax2], ["H", "phase_parameter_1"]):
    ax.hist(x, bins="auto", histtype="step")
    mean = np.mean(x)
    std = np.std(x)
    print(mean, std)
    ax.axvline(mean)
    ax.axvspan(mean - std, mean + std, alpha=0.3)
    ax.set_xlabel(ax_lab)

for x, x_err, ax in zip(["H", "phase_parameter_1"], ["H_err", "phase_parameter_1_err"], [ax1, ax2]):
    x = getattr(pc, x)
    x_err = getattr(pc, x_err)

    if hasattr(x, "unit"):
        x = x.value

    ax.axvline(x, color="k")
    ax.axvspan(x - x_err, x + x_err, alpha=0.3, color="k")

plt.show()

In [ ]:
pc.__dict__

In [ ]:
_pc.__dict__